In [1]:
# Paillier Cryptosystem – Implémentation et Propriétés Homomorphes

## 🔐 Partie 1 : Implémentation du cryptosystème de Paillier

# On commence par importer notre module (paillier.py doit être dans le même dossier)
from paillier import Paillier
from utils import string_to_int, int_to_string
import gmpy2

# Création d'une instance avec des clés de 512 bits (suffisant pour nos tests)
p = Paillier(512)

# Test basique de chiffrement / déchiffrement
message = "Trying to encrypt this message using Paillier"
ciphertext = p.encrypt(string_to_int(message))
decrypted = int_to_string(p.decrypt(ciphertext))

print("Original:", message)
print("Déchiffré:", decrypted)
assert decrypted == message

# ✅ Si l'assert passe, notre implémentation fonctionne

## ✍️ Question 1.2.2 : Que vaut c1 * c2 mod n² ?
# Cela correspond à E(m1 + m2)
m1 = 123
m2 = 456
c1 = p.encrypt(m1)
c2 = p.encrypt(m2)

add_cipher = (c1 * c2) % p.n2
result = p.decrypt(add_cipher)
print("Addition homomorphe:", result, "==", m1 + m2)
assert result == (m1 + m2) % p.n

## ✍️ Question 1.2.3 : Que vaut c1 * g^m2 mod n² ?
# Cela correspond aussi à E(m1 + m2)
add_plain = (c1 * gmpy2.powmod(p.g, m2, p.n2)) % p.n2
result = p.decrypt(add_plain)
print("Addition message clair:", result, "==", m1 + m2)
assert result == (m1 + m2) % p.n

## ✍️ Question 1.2.4 : Que vaut c1^m2 mod n² ?
# Cela correspond à E(m1 * m2)
mul_plain = gmpy2.powmod(c1, m2, p.n2)
result = p.decrypt(mul_plain)
print("Multiplication scalaire:", result, "==", m1 * m2)
assert result == (m1 * m2) % p.n


Original: Trying to encrypt this message using Paillier
Déchiffré: Trying to encrypt this message using Paillier
Addition homomorphe: 579 == 579
Addition message clair: 579 == 579
Multiplication scalaire: 56088 == 56088
